In [11]:
# Install if needed:
# !pip install SPARQLWrapper pandas openpyxl

import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

BASE = "http://127.0.0.1:5000/sparql"

# Map ontology keys → output file stems
TARGETS = {
    "bfo":  "ontology bfo-axioms.xlsx",
    "ies":  "ontology ies-axioms.xlsx",
    "qudt": "ontology qudt-axioms.xlsx",
    "ccom": "ontology ccom-axioms.xlsx",
    "ccot": "ontology ccot-axioms.xlsx",
    "to":   "ontology to-axioms.xlsx",
}

AXIOMS_QUERY = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl:  <http://www.w3.org/2002/07/owl#>
SELECT ?iri ?predicate ?object ?label
WHERE {
  ?iri a ?classType .
  ?iri ?predicate ?object .
  VALUES ?classType { owl:Class rdfs:Class }
  OPTIONAL { ?iri rdfs:label ?label }
  FILTER NOT EXISTS { ?iri owl:deprecated true }
}
ORDER BY ?label
"""

def run_select(endpoint_url: str, query: str) -> pd.DataFrame:
    sp = SPARQLWrapper(endpoint_url)
    sp.setQuery(query)
    sp.setReturnFormat(JSON)
    data = sp.query().convert()
    rows = []
    for b in data["results"]["bindings"]:
        iri = b.get("iri", {}).get("value")
        label = b.get("label", {}).get("value")
        predicate = b.get("predicate", {}).get("value")
        object = b.get("object", {}).get("value")
        axiom = f"{iri} {predicate} {object} ."
        rows.append({"iri": iri,
                     "label": label,
                     "axiom": axiom})
    return pd.DataFrame(rows, columns=["iri", "label", "axiom"])


for key, outfile in TARGETS.items():
    endpoint = f"{BASE}/{key}"
    print(f"Querying {endpoint} …")
    df = run_select(endpoint, AXIOMS_QUERY)
    # Save to Excel
    df.to_excel(outfile, index=False)
    print(f"Saved {len(df)} rows to {outfile}")


Querying http://127.0.0.1:5000/sparql/bfo …
Saved 357 rows to ontology bfo-axioms.xlsx
Querying http://127.0.0.1:5000/sparql/ies …
Saved 2629 rows to ontology ies-axioms.xlsx
Querying http://127.0.0.1:5000/sparql/qudt …
Saved 692 rows to ontology qudt-axioms.xlsx
Querying http://127.0.0.1:5000/sparql/ccom …
Saved 168 rows to ontology ccom-axioms.xlsx
Querying http://127.0.0.1:5000/sparql/ccot …
Saved 216 rows to ontology ccot-axioms.xlsx
Querying http://127.0.0.1:5000/sparql/to …
Saved 364 rows to ontology to-axioms.xlsx


In [10]:




BASE = "http://127.0.0.1:5000/sparql"
endpoint = f"{BASE}/bfo"
df = run_select(endpoint, AXIOMS_QUERY)
df

,iri,label,axiom
0,n34ec83ec5b1f4c768becb67af7a6d6fdb1,None,n34ec83ec5b1f4c768becb67af7a6d6fdb1 http://www...
1,n34ec83ec5b1f4c768becb67af7a6d6fdb1,None,n34ec83ec5b1f4c768becb67af7a6d6fdb1 http://www...
2,n34ec83ec5b1f4c768becb67af7a6d6fdb3,None,n34ec83ec5b1f4c768becb67af7a6d6fdb3 http://www...
3,n34ec83ec5b1f4c768becb67af7a6d6fdb3,None,n34ec83ec5b1f4c768becb67af7a6d6fdb3 http://www...
4,n34ec83ec5b1f4c768becb67af7a6d6fdb2,None,n34ec83ec5b1f4c768becb67af7a6d6fdb2 http://www...
...,...,...,...
352,http://purl.obolibrary.org/obo/BFO_0000148,zero-dimensional temporal region,http://purl.obolibrary.org/obo/BFO_0000148 htt...
353,http://purl.obolibrary.org/obo/BFO_0000148,zero-dimensional temporal region,http://purl.obolibrary.org/obo/BFO_0000148 htt...
354,http://purl.obolibrary.org/obo/BFO_0000148,zero-dimensional temporal region,http://purl.obolibrary.org/obo/BFO_0000148 htt...
355,http://purl.obolibrary.org/obo/BFO_0000148,zero-dimensional temporal region,http://purl.obolibrary.org/obo/BFO_0000148 htt...
